In [1]:
from pathlib import Path
from enum import Enum

import networkx as nx
import numpy as np

In [2]:
input_file = "example.txt"
input = Path(input_file).read_text()
print(input)

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############



In [3]:
WALL = "#"
EMPTY = "."
START = "S"
END = "E"


class Direction(Enum):
    Up = (-1, 0)
    Right = (0, 1)
    Down = (1, 0)
    Left = (0, -1)

    @property
    def index(self) -> int:
        return list(Direction).index(self)


class Rotation(Enum):
    CounterClockwise = -1
    Clockwise = 1


def rotate(direction: Direction, rotation: Rotation) -> Direction:
    directions = list(Direction)
    direction_index = directions.index(direction)
    return directions[(direction_index + rotation.value) % len(directions)]


def move(coordinate: tuple, direction: Direction) -> tuple:
    return tuple(np.array(coordinate) + np.array(direction.value))

In [4]:
maze = np.array([[space for space in line] for line in input.splitlines()])
start = np.argwhere(maze == START)[0]
end = np.argwhere(maze == END)[0]
maze[tuple(start)] = EMPTY
maze[tuple(end)] = EMPTY
print(maze)

[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '.' '.' '#']
 ['#' '.' '#' '.' '#' '#' '#' '.' '#' '.' '#' '#' '#' '.' '#']
 ['#' '.' '.' '.' '.' '.' '#' '.' '#' '.' '.' '.' '#' '.' '#']
 ['#' '.' '#' '#' '#' '.' '#' '#' '#' '#' '#' '.' '#' '.' '#']
 ['#' '.' '#' '.' '#' '.' '.' '.' '.' '.' '.' '.' '#' '.' '#']
 ['#' '.' '#' '.' '#' '#' '#' '#' '#' '.' '#' '#' '#' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '.' '#']
 ['#' '#' '#' '.' '#' '.' '#' '#' '#' '#' '#' '.' '#' '.' '#']
 ['#' '.' '.' '.' '#' '.' '.' '.' '.' '.' '#' '.' '#' '.' '#']
 ['#' '.' '#' '.' '#' '.' '#' '#' '#' '.' '#' '.' '#' '.' '#']
 ['#' '.' '.' '.' '.' '.' '#' '.' '.' '.' '#' '.' '#' '.' '#']
 ['#' '.' '#' '#' '#' '.' '#' '.' '#' '.' '#' '.' '#' '.' '#']
 ['#' '.' '.' '.' '#' '.' '.' '.' '.' '.' '#' '.' '.' '.' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]


## Part I

In [5]:
# coordinate, direction, weight
start_state = (tuple(start), Direction.Right, 0)
queue = [start_state]
visited = {}

while queue:
    coordinate, direction, weight = queue.pop(0)
    clockwise = rotate(direction, Rotation.Clockwise)
    counter_clockwise = rotate(direction, Rotation.CounterClockwise)
    to_check = [
        (move(coordinate, direction), direction, weight + 1),
        (move(coordinate, clockwise), clockwise, weight + 1001),
        (move(coordinate, counter_clockwise), counter_clockwise, weight + 1001),
    ]

    for coordinate, direction, weight in to_check:
        if maze[coordinate] == WALL:
            continue

        if weight < visited.get((coordinate, direction), np.inf):
            visited[(coordinate, direction)] = weight
            queue.append((coordinate, direction, weight))

possible_ends = [
    (possible_end[0], possible_end[1], score)
    for possible_end, score in visited.items()
    if possible_end[0] == tuple(end)
]
possible_ends.sort(key=lambda x: x[2])
min_weight = possible_ends[0][2]
min_weight

7036

In [6]:
# Part I using network X

G = nx.Graph()
start_state = (tuple(start), Direction.Right)
queue = [start_state]

while queue:
    coordinate, direction = queue.pop(0)
    clockwise = rotate(direction, Rotation.Clockwise)
    counter_clockwise = rotate(direction, Rotation.CounterClockwise)
    to_check = [
        (move(coordinate, direction), direction, 1),
        (move(coordinate, clockwise), clockwise, 1001),
        (move(coordinate, counter_clockwise), counter_clockwise, 1001),
    ]

    for new_coordinate, new_direction, weight in to_check:
        if maze[new_coordinate] == WALL:
            continue
        if (new_coordinate, new_direction) not in G.nodes:
            queue.append((new_coordinate, new_direction))
        G.add_edge(
            (coordinate, direction),
            (new_coordinate, new_direction),
            weight=weight,
        )

min_length = np.inf
min_direction = None
for direction in Direction:
    try:
        length = nx.dijkstra_path_length(
            G, (tuple(start), Direction.Right), (tuple(end), direction)
        )
    except Exception:
        continue
    else:
        if length < min_length:
            min_length = length
            min_direction = direction
min_direction, min_length

(<Direction.Up: (-1, 0)>, 7036)

## Part II

In [7]:
len(
    set(
        [
            node[0]
            for path in nx.all_shortest_paths(
                G,
                (tuple(start), Direction.Right),
                (tuple(end), min_direction),
                weight="weight",
                method="dijkstra",
            )
            for node in path
        ]
    )
)

45